# Tag Questions
Tag questions are a form of collaborative dialogue. They encourage the other speaker to engage in the conversation, and ellicite the other speaker's opinion. Because female speakers are traditionally thought to be more collaborative, I hypothesize that female speakers will use more tag questions than male speakers. Also, I believe antagonists will use these less than protagonists--antagonists don't want anyone else's opinion!

Note: The regular expressions below cannot capture sarcasm.

In [1]:
import pandas as pd
import re

In [2]:
movies_df = pd.read_pickle(r"C:/Users/cassi/Desktop/Data_Science/Animated-Movie-Gendered-Dialogue/private/all_tagged_movies_2.pkl")

In [3]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13442 entries, 0 to 13441
Data columns (total 22 columns):
Disney_Period         13442 non-null object
Gender                13442 non-null object
Movie                 13442 non-null object
Role                  13442 non-null object
Song                  13442 non-null object
Speaker               13442 non-null object
Speaker_Status        13442 non-null object
Text                  13442 non-null object
UTTERANCE_NUMBER      13442 non-null int64
Year                  13442 non-null int64
Tokens                13442 non-null object
Types                 13442 non-null object
Token_Count           13442 non-null int64
Type_Count            13442 non-null int64
POS                   13442 non-null object
Tag_Freq              13442 non-null object
Command_Count         13442 non-null int64
Top_Tag               13442 non-null object
Adj_Count             13442 non-null int64
Adj_over_Tokens       13442 non-null float64
Interjection_Cou

In [7]:
tag_questions = ['you?', "you ?", "you think?", "think so?", "you suppose?", "you suppose so?", "yes?", "no?", 
                 "right?", "correct?", "it?", "he?", "she?", "we?", "i?","you think ?", "you think so ?", "you suppose ?", "you suppose so ?", "yes ?", "no ?", 
                 "right ?", "correct ?", "it ?", "he ?", "she ?", "we ?", "i ?"]

In [19]:
tag_lines = []
for line in movies_df.Text:
    for tag in tag_questions:
        if tag in line:
            tag_lines.append(line)

In [20]:
len(tag_lines)

438

In [21]:
len(set(tag_lines))

403

In [50]:
tag_lines_3 = []
for line in movies_df.Text:
    tags = re.findall(r" (you|he|she|we|i|it|think|suppose|think so|suppose so|yes|no|right|correct|okay|alright) *\?", line)
    num_tags = len(tags)
    if num_tags >= 1:
        tag_lines_3.append(line)

In [22]:
len(tag_lines_2)

429

In [51]:
len(tag_lines_3)

437

In [52]:
set(tag_lines_3)

{'... two of the finest dragon riders west of luk tuk. that oughtta make the boss happy, right?',
 "a snake, am i? perhaps you'd like to see how snakelike i can be! ",
 "abu! are you okay? i'm sorry, abui made a mess of everything, somehow. i gotta go back and set things right. ",
 'ah, you okay?',
 'al? al! kid, snap out of it! you can\'t cheat on this one! i can\'t help you unless you make a wish. you have to say "genie i want you to save my life." got it? okay. c\'mon aladdin!! ',
 'all right, what is it? ',
 'all right, you get the gist. she sings to it, she turns young, creepy, right?',
 "all this time, you took after me. and where was i? i'm so sorry, hiccup. can we start over? will you give me another chance?",
 "alright, be strong babies! be strong. now, coco, peanut, you listen to mama, alright? and bananas, no more roastin' marshmallows on your sister's head.",
 "alright. i'm off to bed. you should be too. tomorrow we get into the big boys. slowly but surely making our way up

In [38]:
tag_lines_3[:10]

["did you see it ? how big is it ? was it a dragon ? has it got horns ? was it breathing fire ? was it droolin' ? what was it doin' ? ",
 'what is it ? ',
 "and, you ? you're sleepy. ",
 'and you ? ',
 'kill you ? ',
 "come on. let's see them. oh, bashful, my, my, my ! and you ? tsk, tsk, tsk ! ",
 "we ain't gonna do it, are we ? ",
 'was the princess you ? ',
 'like it, gusgus? like it? like it? ',
 "suspect! bah! look, the boy's coming home today isn't he? "]

In [25]:
set(tag_lines_2) - set(tag_lines)

{"between you and me, the village could do with a little less feeding, don't ya think?",
 "have you been to insectopia?  have  you  ?  no, ya goddam larvas!  but i have...  ...mosquitos n' caterpillars n' beetles -- all livin' in peace, stuffin their guts with food...no rules, no regulations...you can be your own ant there...  it's insectopia! insectopia  !",
 'no, do ya think?',
 "princess, has it ever occurred to you that they're not going to rescue you  ?",
 'um...i think so?',
 'you cut me deep, shrek. you cut me real deep just now. you know what i think? i think this whole wall thing is just a way to keep somebody out.',
 "you know what i think? i think we just dodged a bullet. i mean, what's this clown know about bringing joy to children anyway?",
 'you really think so?'}

In [24]:
len(set(tag_lines_2))

411

In [53]:
def find_tags(line):
    tags = re.findall(r" (you|he|she|we|i|it|think|suppose|think so|suppose so|yes|no|right|correct|alright|okay) *\?", line)
    num_tags = len(tags)
    return num_tags

In [54]:
movies_df['Tags'] = movies_df.Text.map(find_tags)

In [55]:
movies_df.Tags.describe()

count    13442.000000
mean         0.034221
std          0.194075
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          5.000000
Name: Tags, dtype: float64

In [56]:
movies_df[movies_df.Tags == 5]

,Disney_Period,Gender,Movie,Role,Song,Speaker,Speaker_Status,Text,UTTERANCE_NUMBER,Year,...,Type_Count,POS,Tag_Freq,Command_Count,Top_Tag,Adj_Count,Adj_over_Tokens,Interjection_Count,Inter_over_Tokens,Tags
4761,LATE,f,Brave,PRO,D,merida,NON-P,"can i? can i? can i? can i? please, can i?",7,2012,...,5,"[(can, MD), (i, VB), (?, .), (can, MD), (i, VB...","{'MD': 5, 'VB': 5, '.': 5, 'NN': 1, ',': 1}",5,MD,0,0.0,0,0.0,5


In [57]:
movies_df.groupby('Gender')['Tags'].describe()

,count,mean,std,min,25%,50%,75%,max
Gender,,,,,,,,
f,4216.0,0.034867,0.199569,0.0,0.0,0.0,0.0,5.0
m,8914.0,0.034440,0.192542,0.0,0.0,0.0,0.0,3.0
n,312.0,0.019231,0.159225,0.0,0.0,0.0,0.0,2.0


In [58]:
movies_df.groupby('Role')['Tags'].describe()

,count,mean,std,min,25%,50%,75%,max
Role,,,,,,,,
ANT,2037.0,0.027982,0.173665,0.0,0.0,0.0,0.0,2.0
HELPER,3549.0,0.033249,0.195839,0.0,0.0,0.0,0.0,3.0
N,2094.0,0.026743,0.170021,0.0,0.0,0.0,0.0,2.0
PRO,5762.0,0.039743,0.207438,0.0,0.0,0.0,0.0,5.0


In [ ]:
#no tests run yet, but it's looking like there's a pretty big diff between antagonists and protagonists using tags...